I am going to test three different machine learning algorithms, a random forest classifier, a gradient boosting classifier, and an adaboost classifier.  The reason that I am choosing these is that they tend to perform well on classification tasks without excessive computation time, as would likely be the case in using SVMs.  They also do not typically need as much data to perform well as some other techniques, particularly neural networks.  One of the biggest downsides to using these classifiers is that they do not handle nans or text data easily, which is why I used naive bayes on the text columns as a work around.  Additionally, one can make feature importance plots using these algorithms, which could be useful if Taarifa would like to have a sense of what is likely to predict a breakdown.

In [48]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split

np.random.seed(42)

In [49]:
train_df = pd.read_csv('../data/processed/clean_train_df.csv')
test_df = pd.read_csv('../data/processed/clean_test_df.csv')
train_df.drop('date_recorded', axis=1, inplace=True)
train_df.info()

train_y = train_df.pop('status_group').values
train_X = train_df.values
test_X = test_df.values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 12 columns):
Unnamed: 0           59400 non-null int64
id                   59400 non-null int64
amount_tsh           59400 non-null float64
gps_height           59400 non-null int64
longitude            59400 non-null float64
latitude             59400 non-null float64
num_private          59400 non-null int64
public_meeting       59400 non-null bool
permit               59400 non-null bool
construction_year    59400 non-null float64
status_group         59400 non-null object
non functional       59400 non-null float64
dtypes: bool(2), float64(5), int64(4), object(1)
memory usage: 4.6+ MB


In [50]:
model_train_X, model_valid_X, model_train_y, model_valid_y = train_test_split(train_X, train_y)

In [51]:
model_rf = RandomForestClassifier()
model_rf.fit(model_train_X, model_train_y)
model_rf.score(model_valid_X, model_valid_y)

0.74410774410774416

In [52]:
model_gb = GradientBoostingClassifier()
model_gb.fit(model_train_X, model_train_y)
model_gb.score(model_valid_X, model_valid_y)

0.75218855218855218

In [53]:
model_ab = AdaBoostClassifier()
model_ab.fit(model_train_X, model_train_y)
model_ab.score(model_valid_X, model_valid_y)

0.74922558922558924

Gb w/o construction_year = .745